In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install gradio

     |████████████████████████████████| 1.1MB 9.9MB/s 
     |████████████████████████████████| 215kB 36.1MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 962kB 34.7MB/s 
     |████████████████████████████████| 3.2MB 47.9MB/s 
  Created wheel for Flask-BasicAuth: filename=Flask_BasicAuth-0.2.0-cp37-none-any.whl size=4232 sha256=a2e4f0e257d2a4a0142727da63eef3561a2fcb4e6d5fe12d2451a98060f7318f
  Stored in directory: /root/.cache/pip/wheels/8a/d5/8c/1f40bffc038e6ab2865836cc303e312e16de7b6b577be16b25
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp37-none-any.whl size=4710 sha256=c05652d955081c5e4bbfd84622d75f261713ce64610a65ddb08cba92c2a767eb
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp37-none-any.whl size=3372 sha256=2b5ab6a01f004c1480885be03acbda14d18fb42e3cb0df4358157a7ebb644bda
  Stored in director

In [3]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle

import tensorflow as tf


fold = 5
num_classes = 20

with open('drive/MyDrive/ProtSeq_pred/models/tokenizerndr.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)
#for i in range(fold):
model = [tf.keras.models.load_model(f"drive/MyDrive/ProtSeq_pred/models/model{i}.h5") for i in range(fold)]

def seq_pred(sequence):
  # Inference 

  data = [sequence]
  xtest = tokenizer.texts_to_sequences(data)
  xtest = tf.keras.preprocessing.sequence.pad_sequences(xtest,padding='post',
                                        truncating='post',maxlen=300)
  final_preds = []                                    
  for i in range(fold):
    loaded_model = model[i]
    preds = loaded_model.predict(xtest)
    final_preds.append(preds)

  pred_0 = np.concatenate(final_preds,axis=0)
  n_pred = np.reshape(pred_0,(fold,len(data),num_classes))
  avg = np.mean(n_pred,axis=0)
  idx2class = {0:'class0',1:'class1',2:'class10',3:'class11',4:'class12',5:'class13',6:'class14',7:'class15',
             8:'class16',9:'class17',10:'class18',11:'class19',12:'class2',13:'class3',
             14:'class4',15:'class5',16:'class6',17:'class7',18:'class8',19:'class9'}
  #y_classes = np.argmax(avg,axis=1)
  #final = idx2class[y_classes.item()]
  final_predictions = np.concatenate(avg,axis=0)
  return {idx2class[i]: float(final_predictions[i]) for i in range(num_classes)}

iface = gr.Interface(fn=seq_pred, inputs=gr.inputs.Textbox(placeholder="Enter Protein sentence here..."), outputs=gr.outputs.Label(num_top_classes=3))
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://10317.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://10317.gradio.app')